# Save, Load and Export Models in Keras

## Task 1: Import Libraries

In [1]:
import tensorflow as tf
import numpy as np
import os

print('This notebook works with TensorFlow version:', tf.__version__)

folders = ['tmp', 'models', 'model_name', 'weights']
for folder in folders:
    if not os.path.isdir(folder):
        os.mkdir(folder)

print(os.listdir('.'))

This notebook works with TensorFlow version: 2.7.0
['.config', 'models', 'tmp', 'weights', 'model_name', 'sample_data']


## Task 2: Create Model

In [2]:
def create_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=(784,)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
    return model

model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               100480    
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dense_2 (Dense)             (None, 10)                1290      
                                                                 
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________


## Task 3: Data Preprocessing

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

x_train = np.reshape(x_train, (x_train.shape[0], 784))/255.
x_test = np.reshape(x_test, (x_test.shape[0], 784))/255.

y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

4431872/4422102 [==============================] - 0s 0us/step


## Task 4: Model Checkpoint During Training

In [4]:
checkpoint_dir = 'weights/'

_ = model.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    epochs=2,
    batch_size=512,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(
            os.path.join(checkpoint_dir, 'epoch_{epoch:02d}_acc_{val_acc:.4f}'),
            monitor='val_acc', save_weights_only=True, save_best_only=True
        )
    ]
)

Epoch 1/2
118/118 [==============================] - 3s 8ms/step - loss: 0.6970 - acc: 0.7641 - val_loss: 0.4985 - val_acc: 0.8241
Epoch 2/2
118/118 [==============================] - 1s 6ms/step - loss: 0.4380 - acc: 0.8469 - val_loss: 0.4260 - val_acc: 0.8484


In [5]:
os.listdir(checkpoint_dir)

['checkpoint',
 'epoch_02_acc_0.8484.data-00000-of-00001',
 'epoch_02_acc_0.8484.index',
 'epoch_01_acc_0.8241.data-00000-of-00001',
 'epoch_01_acc_0.8241.index']

## Task 5: Load Weights

In [6]:
model = create_model()
print(model.evaluate(x_test, y_test, verbose=False))

[2.3507182598114014, 0.09600000083446503]


In [9]:
model.load_weights('weights/epoch_02_acc_0.8484')
print(model.evaluate(x_test, y_test, verbose=False))

[0.4260275959968567, 0.8483999967575073]


## Task 6: Saving Complete Model During Training

In [10]:
models_dir = 'models'

model = create_model()

_ = model.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    epochs=2,
    batch_size=512,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(
            os.path.join(models_dir, 'epoch_{epoch:02d}_acc_{val_acc:.4f}.h5'),
            monitor='val_acc', save_weights_only=False, save_best_only=False
        )
    ]
)

Epoch 1/2
118/118 [==============================] - 1s 7ms/step - loss: 0.7071 - acc: 0.7614 - val_loss: 0.4987 - val_acc: 0.8225
Epoch 2/2
118/118 [==============================] - 1s 5ms/step - loss: 0.4351 - acc: 0.8480 - val_loss: 0.4543 - val_acc: 0.8379


In [11]:
os.listdir(models_dir)

['epoch_02_acc_0.8379.h5', 'epoch_01_acc_0.8225.h5']

## Task 7: Load Models

In [12]:
model = create_model()
print(model.evaluate(x_test, y_test, verbose=False))

[2.4234790802001953, 0.10050000250339508]


In [14]:
model = tf.keras.models.load_model('models/epoch_02_acc_0.8379.h5')
model.summary()
print(model.evaluate(x_test, y_test, verbose=False))

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 128)               100480    
                                                                 
 dense_7 (Dense)             (None, 128)               16512     
                                                                 
 dense_8 (Dense)             (None, 10)                1290      
                                                                 
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________
[0.454302042722702, 0.8378999829292297]


## Task 8: Manually Saving Weights and Models

In [15]:
model.save_weights('tmp/manually_saved')
print(os.listdir('tmp'))

['manually_saved.index', 'checkpoint', 'manually_saved.data-00000-of-00001']


In [16]:
model.save('tmp/manually_saved_model.h5')
print(os.listdir('tmp'))

['manually_saved.index', 'checkpoint', 'manually_saved.data-00000-of-00001', 'manually_saved_model.h5']


## Task 9: Exporting and Restoring SavedModel Format

In [17]:
model.save('model_name')
print(os.listdir('model_name'))

INFO:tensorflow:Assets written to: model_name/assets
['variables', 'keras_metadata.pb', 'saved_model.pb', 'assets']


In [18]:
model = tf.keras.models.load_model('model_name')
print(model.evaluate(x_test, y_test, verbose=False))

[0.454302042722702, 0.8378999829292297]
